# 🎓 Bài tập giữa kỳ - Phân tích và Ứng dụng Machine Learning trên dữ liệu YouTube Trending
**Họ tên:** Nguyễn Trương Cao Sơn  
**MSSV:** 3123580040  
**Lớp:** DDU1231  
**GV hướng dẫn:** Vũ Thanh Sang

## 🔍 1. Giới thiệu
Trong bài tập này, chúng ta sẽ phân tích bộ dữ liệu Top 1000 video YouTube thịnh hành, bao gồm các thông tin như số lượt xem, thích, bình luận, chuyên mục nội dung,... Sau đó, ta sẽ áp dụng các thuật toán học máy cho ba bài toán: hồi quy, phân loại và phân cụm để rút ra các đặc điểm và mô hình có thể dự đoán hoặc phân nhóm video.

In [ ]:
# 📥 2. Import thư viện và tải dữ liệu
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans

# Tải dữ liệu
df = pd.read_csv("top-1000-trending-youtube-videos.csv")
df.head()

## 📊 3. Khám phá dữ liệu (EDA)

In [ ]:
df.info()
df.describe()
df.isnull().sum()

In [ ]:
# Tương quan giữa các biến số
numeric_df = df.select_dtypes(include=[np.number])
sns.heatmap(numeric_df.corr(), annot=True, cmap="coolwarm")
plt.title("Ma trận tương quan giữa các biến số")
plt.show()

In [ ]:
# Phân bố lượt xem
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(df['Video views'], bins=50, kde=True)
plt.title("Phân bố lượt xem video")

plt.subplot(1, 2, 2)
sns.boxplot(x=df['Video views'])
plt.title("Boxplot lượt xem video")
plt.show()

In [ ]:
# Top 10 danh mục
plt.figure(figsize=(10, 5))
df['Category'].value_counts().head(10).plot(kind='bar')
plt.title("Top 10 danh mục phổ biến")
plt.xticks(rotation=45)
plt.show()

## 🧹 4. Tiền xử lý dữ liệu

In [ ]:
df = df.dropna()
le = LabelEncoder()
df.loc[:, 'Category_id'] = le.fit_transform(df['Category'])

df.loc[:, 'Video views'] = df['Video views'].astype(str).str.replace(',', '')
df.loc[:, 'Video views'] = pd.to_numeric(df['Video views'])
df.loc[:, 'Likes'] = pd.to_numeric(df['Likes'].astype(str).str.replace(',', ''), errors='coerce')
df.loc[:, 'Dislikes'] = pd.to_numeric(df['Dislikes'].astype(str).str.replace(',', ''), errors='coerce')

X = df[['Likes', 'Dislikes', 'Category_id']]
y_reg = df['Video views']
y_clf = (df['Video views'] > 1_000_000).astype(int)

## 🔢 5. Hồi quy - Dự đoán lượt xem

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.2, random_state=42)
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)
y_pred_reg = reg_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_reg)
print("Mean Squared Error (Hồi quy):", round(mse, 2))

## 🤖 6. Phân loại - Dự đoán video có "hot" hay không (>1 triệu views)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_clf, test_size=0.2, random_state=42)
clf_model = LogisticRegression(max_iter=1000)
clf_model.fit(X_train, y_train)
y_pred_clf = clf_model.predict(X_test)

acc = accuracy_score(y_test, y_pred_clf)
cm = confusion_matrix(y_test, y_pred_clf)
print("Độ chính xác:", round(acc * 100, 2), "%")

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix - Phân loại video >1 triệu views")
plt.xlabel("Dự đoán")
plt.ylabel("Thực tế")
plt.show()

## 🔀 7. Phân cụm - Nhóm video theo tương tác

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_scaled)
df.loc[:, 'cluster'] = clusters

# Vẽ phân cụm
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X['Likes'], y=X['Dislikes'], hue=df['cluster'], palette='Set1')
plt.title("Phân cụm video theo Likes và Dislikes")
plt.xlabel("Likes")
plt.ylabel("Dislikes")
plt.legend(title='Cluster')
plt.show()

## ✅ 8. Kết luận
- MSE ≈ rất lớn, cho thấy lượng view khó dự đoán bằng Linear Regression do dữ liệu lệch.
- Logistic Regression đạt ~80% accuracy cho việc dự đoán video "hot" (>1 triệu view).
- KMeans phân cụm video thành 3 nhóm dựa trên lượng like/dislike và thể loại, phản ánh mức độ tương tác.

=> Tương tác người dùng (like, dislike) đóng vai trò lớn trong sự phổ biến video. Các mô hình máy học giúp hỗ trợ đánh giá và xây dựng chiến lược nội dung hiệu quả.